In [ ]:
import os
import scipy
import numpy as np
import trimesh
from trimesh import viewer

In [ ]:
mesh1 = trimesh.base.Trimesh(vertices=np.loadtxt('data/wolf0.vert'), faces=np.loadtxt('data/wolf0.tri', dtype=np.int32)-1)
mesh2 = trimesh.base.Trimesh(vertices=np.loadtxt('data/wolf1.vert'), faces=np.loadtxt('data/wolf1.tri', dtype=np.int32)-1)

mesh1.visual.vertex_colors = trimesh.visual.random_color()

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html((trimesh.util.concatenate([mesh1, mesh2])).scene())
with open("scene.html", "w") as file:
    file.write(html)

In [ ]:
def get_laplacian(faces):
    N = faces.max() + 1
    # face to edge
    row, col = np.concatenate([faces[:2], faces[1:], faces[::2]], axis=1)
    # to undirected
    row, col = np.unique(np.concatenate([np.stack([row, col]), np.stack([col, row])], axis=1), axis=1)
    # edge weight
    data = np.ones(len(row))
    # adjacency matrix
    A = scipy.sparse.coo_matrix((data, (row, col)), shape=(N, N)).todok().tocsr()
    # ensure symmetry
    assert (abs(A - A.T) > 1e-10).nnz == 0
    # degree matrix
    D = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten())
    # combinatorial Laplacian
    L = D - A
    return L.tocoo()

In [ ]:
def arap(source, target, num_iter=100):
    x = source.vertices
    y = target.vertices
    N = len(x)
    L = get_laplacian(source.faces.T)
    for i in range(num_iter):
        print('iteration: ', i + 1)
        # measure rms error
        print('rms error: ', np.sqrt(np.mean(np.linalg.norm(x - y, axis=1) ** 2)))
        # calculate the S-matrix
        S = np.einsum('ij,ik->ijk', x[L.row] - x[L.col], y[L.row] - y[L.col])
        S = scipy.sparse.bsr_matrix((S, L.tocsr().indices, L.tocsr().indptr),
                                    shape=(3*N, 3*N))
        S = np.asarray(S.reshape(N, N*3*3).sum(axis=0))
        S = S.reshape(3, N, 3).swapaxes(0, 1)
        # singular value decomposition
        U, _, Vh = np.linalg.svd(S)
        # calculate the rotation matrices
        R = np.einsum('ikj, ilk -> ijl', Vh, U)
        # calculate the b vector
        b = np.einsum('ijk,ik->ij', R[L.row] + R[L.col], x[L.row] - x[L.col])[:, None] * -0.5
        b = scipy.sparse.bsr_matrix((b, L.tocsr().indices, L.tocsr().indptr),
                                    shape=(N, 3*N))
        b = np.asarray(b.sum(axis=0))
        b = b.reshape(N, 3)
        # solve the system of linear equations using conjugate gradient method
        x_solution, info = scipy.sparse.linalg.cg(L, b[:, 0])
        print('info: ', info)
        y_solution, info = scipy.sparse.linalg.cg(L, b[:, 1])
        print('info: ', info)
        z_solution, info = scipy.sparse.linalg.cg(L, b[:, 2])
        print('info: ', info)
        x = np.stack([x_solution, y_solution, z_solution], axis=1)
        # # rigid registration - this is optional, I had to use it for my project
        # U, _, Vh = np.linalg.svd(np.einsum('ij, ik -> jk', x - x.mean(axis=0), y - y.mean(axis=0)))
        # R = np.einsum('ji, kj -> ik', Vh, U)
        # t = y.mean(axis=0) - R @ x.mean(axis=0)
        # x = x @ R + t
    return x


In [ ]:
solution = arap(mesh1, mesh2)

In [ ]:
trans_mesh = trimesh.base.Trimesh(solution, mesh1.faces)

trans_mesh.visual.vertex_colors = trimesh.visual.random_color()

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html(trimesh.util.concatenate([trans_mesh, mesh2]).scene())
with open("scene.html", "w") as file:
    file.write(html)

# THE TWO MESHES SHOULD BE ALIGNED NOW